In [5]:
import requests
from bs4 import BeautifulSoup as bs
import pandas as pd

In [6]:
def get_pagenum(url):
    req = requests.get(url)
    bsoup = bs(req.content, 'html.parser')
    pages = []
    page_num = bsoup.find('div', {'id': 'searchCountPages'}).text.strip()
    for num in page_num.split():
        if num.isdigit():
            pages.append(num)
    return int(pages[1])

In [7]:
def get_html():
    header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/101.0.4951.41 Safari/537.36'}
    url = f'https://www.indeed.com/jobs?q=data%20analyst%20entry%20level&l=remote&start=0&vjk=ff725c70949a666a'
    r = requests.get(url, header)
    soup = bs(r.content, 'html.parser')
    return soup
    

In [12]:
def get_data(soup):
    divs = soup.find_all('div', class_='job_seen_beacon')
    for item in divs:
        title = item.find('h2').text.strip()
        company = item.find('span', class_='companyName').text.strip()
        try:
            salary = item.find('div', class_='attribute_snippet').text.strip()
        except:
            salary = ''
        summary = item.find('div', class_='job-snippet').text.strip().replace('\n', ' ')
        job = {'title': title, 'company': company, 'salary': salary, 'summary': summary}
        jobs.append(job)
    return

In [17]:
url = 'https://www.indeed.com/jobs?q=data%20analyst%20entry%20level&l=remote&start=0&vjk=ff725c70949a666a'
page = get_pagenum(url)
jobs = []
for i in range(0, page, 10):
    soup = get_html()
    get_data(soup)
data = pd.DataFrame(jobs)
data.to_csv('indeed_jobs.csv')
len(jobs)

105

In [16]:
data.head()

,title,company,salary,summary
0,Business/Data Analyst (Entry Level),Agama Solutions,$30 an hour,Role: Business/Data Analyst (Entry Level). 1+ ...
1,newBusiness Analyst (Entry Level),Agama Solutions,$30 - $35 an hour,Troubleshoot data issues by validating differe...
2,Talent Development Data Analyst,Southern New Hampshire University,Full-time,Lead the descriptive analytics for all Talent ...
3,Jr. Business Analyst,Kajeet,Full-time,They need to be able to pull relevant data fro...
4,"newData Solutions Analyst, US Analytics",Liberty Mutual Insurance,"$70,000 - $100,000 a year",Ensures data solutions include deliverables re...
